# Graphic User Interface or GUI: Gradio!

In this notebook you will learn a new tool called Gradio! This tool will allow you to create a chatbot using a portion of the code you already used in the previous activity.

Gradio is an open-source Python library designed to simplify the process of creating customizable UI components for machine learning models. It allows data scientists, developers, and researchers to quickly build and share web applications or demos of their machine learning models without extensive knowledge of web development.

With Gradio, you can create interfaces for a wide range of applications, including image classification, text generation, and more. It supports various input and output types like text, images, videos, and audio, making it versatile for showcasing different types of machine learning models.

One of the key features of Gradio is its ease of use. You can set up a basic interface with just a few lines of code, and it integrates seamlessly with popular machine learning libraries such as TensorFlow, PyTorch, and Hugging Face Transformers. This integration makes it easy to create demos for pre-trained models or models you've developed yourself.

Gradio is also designed with sharing and collaboration in mind. Once an interface is created, it can be shared through a link, embedded in a website, or even integrated into Jupyter notebooks, like the one you are using right now. This makes it an excellent tool for educators, researchers, and developers looking to showcase their work or collaborate with others.

---

Dans ce cahier, vous apprendrez à utiliser un nouvel outil appelé Gradio ! Cet outil vous permettra de créer un chatbot en utilisant une partie du code que vous avez déjà utilisé dans l'activité précédente.

Gradio est une bibliothèque Python open-source conçue pour simplifier le processus de création de composants d'interface utilisateur personnalisables pour les modèles d'apprentissage automatique. Elle permet aux scientifiques de données, développeurs et chercheurs de construire rapidement et partager des applications web ou des démos de leurs modèles d'apprentissage automatique sans avoir une connaissance approfondie du développement web.

Avec Gradio, vous pouvez créer des interfaces pour une large gamme d'applications, y compris la classification d'images, la génération de texte, et plus encore. Elle prend en charge divers types d'entrées et de sorties comme le texte, les images, les vidéos et l'audio, ce qui la rend polyvalente pour présenter différents types de modèles d'apprentissage automatique.

L'une des caractéristiques clés de Gradio est sa facilité d'utilisation. Vous pouvez configurer une interface de base avec seulement quelques lignes de code, et elle s'intègre parfaitement avec des bibliothèques d'apprentissage automatique populaires telles que TensorFlow, PyTorch, et Hugging Face Transformers. Cette intégration facilite la création de démos pour des modèles pré-entraînés ou des modèles que vous avez développés vous-même.

Gradio est également conçue avec le partage et la collaboration à l'esprit. Une fois une interface créée, elle peut être partagée via un lien, intégrée dans un site web, ou même intégrée dans des notebook Jupyter comme celui-là que tu es en train d'utiliser. Cela en fait un excellent outil pour les éducateurs, les chercheurs et les développeurs cherchant à présenter leur travail ou à collaborer avec d'autres.

# Let's explore Gradio

In the first cell We will first of all install the package gradio and import it here.

Then in the second cell we will create the function that Gradio needs to have as action to execute, in our example here it will be function hello_user.

Finally we will launch the interface! To launch the interface, we need only to call it gr.Interface and specify that the input will be a text (our name for example!), and we expect an output as a text and the function to be run

Let's try it out!

---

Dans la première cellule, nous allons tout d'abord installer le paquet gradio et l'importer ici.

Ensuite, dans la deuxième cellule, nous allons créer la fonction dont Gradio a besoin comme action à exécuter, dans notre exemple ici ce sera la fonction hello_user.

Enfin, nous lancerons l'interface ! Pour lancer l'interface, il suffit de l'appeler avec gr.Interface et de spécifier qu'on s'attend à une entrée en tant que texte (peut-etre votre prénom !), une sortie en tant que texte et la fonction à exécuter.

Essayons !

In [ ]:
! pip install gradio
import gradio as gr

In [2]:
def hello_user(input):
  return("Hey! " + input + ", Take care! ")

In [ ]:
gr.Interface(inputs = gr.Text(), outputs = gr.Text(), fn = hello_user).launch()

# And now it's time to use our code / C'est le moment d'utiliser notre code !

Similarly to the first activity we would need to import some packages and dependencies to make our code work.

---

De manière similaire à la première activité, nous devrons importer certains packages et dépendances pour que notre code fonctionne.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import warnings
warnings.filterwarnings('ignore')

!pip install torch transformers accelerate

and also our model!

---

et aussi notre modèle !

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread

tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1", torch_dtype=torch.float16)

model = model.to('cuda:0')

# Some modification of our previous code / Quelques modifications à faire au code précedent

Since we are going to build our chatbot we need to add 2 new ingredients to our recipe:


1.   a way to tell to the model that when it meets some characters it needs to stop - this will be done through the *class StopOnTokens()*
2.   a way to let the model have in memory what we have said previously - this will be done inside the function *def predict()*

Of course there are some limitations! The model we are using is for the English language and it is not the biggest model in the market, so probably we won't be able to ask it anything we want without encountering some... surprises! (or hallucination!)

Similarly to the previous activity, use your - not artificial - intelligence and be responsible and stay curious by playing with it.

---

Puisque nous allons construire notre chatbot, nous devons ajouter 2 nouveaux ingrédients à notre recette :

1. un moyen de dire au modèle que, lorsqu'il rencontre certains caractères, il doit s'arrêter - cela sera réalisé grâce à la *class StopOnTokens()*
2. un moyen de permettre au modèle de se souvenir de ce que nous avons dit précédemment - cela sera fait à l'intérieur de la fonction *def predict()*

Bien sûr, il y a des limites ! Le modèle que nous utilisons est pour la langue anglaise et ce n'est pas un grand modèle, donc probablement nous ne pourrons pas lui demander tout ce que nous voulons sans rencontrer quelques... surprises ! (ou hallucinations !)

De manière similaire à l'activité précédente, utilisez votre intelligence - non artificielle - et soyez responsable et restez curieuses en jouant avec.

# How to make the model stop the text generation / Comment arreter la génération du texte.

The class StopOnTokens is designed to be a criterion for stopping the text generation process. This is useful in scenarios where you want the generation to stop if certain conditions are met, for example, when a specific token/word or set of tokens/words is generated.

The class defines a list stop_ids containing token IDs (in this case, [29, 0]) that, when generated, should cause text generation to stop.
These token IDs would correspond to specific conditions or characters (like end-of-text).

It then iterates over each stop_id in stop_ids. For the current set of generated tokens (input_ids), it checks if the last token generated (input_ids[0][-1]) is equal to the stop_id.
If any of the stop_ids match the last generated token, the method returns True, signaling that the stopping condition has been met and text generation should stop.
If none of the stop_ids match the last generated token, it returns False, indicating that text generation should continue.

---

La classe StopOnTokens est conçue pour être un critère d'arrêt du processus de génération de texte. Cela est utile dans les scénarios où vous souhaitez que la génération s'arrête si certaines conditions sont remplies, par exemple, lorsqu'un token/mot spécifique ou un ensemble de tokens/mots est généré.

La classe définit une liste stop_ids contenant des IDs de tokens (dans ce cas, [29, 0]) qui, lorsqu'ils sont générés, devraient provoquer l'arrêt de la génération de texte.
Ces IDs de token correspondraient à des conditions ou des caractères spécifiques (comme endoftext).

Elle itère ensuite sur chaque stop_id dans stop_ids. Pour l'ensemble actuel de tokens générés (input_ids), elle vérifie si le dernier token généré (input_ids[0][-1]) est égal au stop_id.
Si l'un des stop_ids correspond au dernier token généré, la méthode retourne True, signalant que la condition d'arrêt a été atteinte et que la génération de texte doit s'arrêter.
Si aucun des stop_ids ne correspond au dernier token généré, elle retourne False, indiquant que la génération de texte doit continuer.

In [6]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [29, 0] #< ,  <|endoftext|>
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

# Predict Function / Fonction predict


This code defines a function named *predict* that generates responses for a chatbot based on a given message and the conversation history. The process involves a series of steps to prepare the input, generate a response using a language model, and yield the response incrementally.

Let's break it down!

# 1 History Transformation

*history_transformer_format = history + [[message, ""]]*: This line formats the conversation history by appending the new message to the end of the history. Each entry in the history is expected to be a list of two strings: one from the human and the response from the bot. The new message is appended without a corresponding bot response (yet).
# 2 the new Stop condition
*stop = StopOnTokens()*: This is the implementation of what we have seen before for the stopping criteria!

# 3 Formatting Input
Messages from the history_transformer_format are concatenated into a single string, messages, formatted with special markers (\n<human>: and \n<bot>:) to differentiate between messages from the human and responses from the bot. This will allows the user to talk with the model without writing the roles and so.. Be more natural!

# 4 Preparing model input
*model_inputs = tokenizer([messages], return_tensors="pt").to("cuda"):* Tokenizes the concatenated messages, converting them into a numerical format that the model can understand (model_inputs)

# 5 Streaming and Generation Setup

Initializes a TextIteratorStreamer, an object that manages streaming output tokens from the model. This is used because we are in a chatbot environment so the Streamer stores print-ready text in a queue, to be used by a downstream application as an iterator.

generate_kwargs is a dictionary that aggregates arguments for the generation function, including some of the parameters that you already used like max_new_tokens, do_sample, top_p, top_k, temperature, num_beams, and the stopping_criteria.

# 6 Generation Execution:

A Thread is started targeting the model's generate method with the arguments prepared in generate_kwargs. This asynchronous approach allows the function to yield partial messages as they're being generated without waiting for the entire generation process to complete.

#7 Yielding Partial Responses:

The function iterates over new tokens from the streamer. As long as the new token is not the special character <, it concatenates these tokens to partial_message and yields the partial message. This allows the function to provide real-time updates of the message being generated, useful for applications that want to show the user that the bot is "typing" or gradually revealing its response.

----

# Fonction Predict / Fonction predict

Ce code définit une fonction nommée *predict* qui génère des réponses pour un chatbot basé sur un message donné et l'historique de la conversation. Le processus implique une série d'étapes pour préparer l'entrée, générer une réponse en utilisant un modèle de langage, et fournir la réponse de manière incrémentale.

Décomposons cela !

# 1 Transformation de l'historique

*history_transformer_format = history + [[message, ""]]* : Cette ligne formate l'historique de la conversation en ajoutant le nouveau message à la fin de l'historique. Chaque entrée dans l'historique est supposée être une liste de deux chaînes : une de l'humain et la réponse du bot. Le nouveau message est ajouté sans une réponse correspondante du bot (pour l'instant).
# 2 La nouvelle condition d'arrêt
*stop = StopOnTokens()* : C'est la mise en œuvre de ce que nous avons vu précédemment pour les critères d'arrêt !

# 3 Formatage de l'entrée
Les messages de l'history_transformer_format sont concaténés en une seule chaîne, messages, formatée avec des marqueurs spéciaux (\n<human> : et \n<bot> :) pour différencier les messages de l'humain et les réponses du bot. Cela permettra à l'utilisateur de parler avec le modèle sans écrire les rôles et donc... Être plus naturel !

# 4 Préparation de l'entrée du modèle
*model_inputs = tokenizer([messages], return_tensors="pt").to("cuda") :* Tokenise les messages concaténés, les convertissant en un format numérique que le modèle peut comprendre (model_inputs)

# 5 Configuration du Streaming et de la Génération

Initialise un TextIteratorStreamer, un objet qui gère le flux de tokens de sortie du modèle. Ceci est utilisé parce que nous sommes dans un environnement de chatbot, donc le Streamer stocke le texte prêt à être imprimé dans une file d'attente, pour être utilisé par une application en aval comme un itérateur.

generate_kwargs est un dictionnaire qui agrège les arguments pour la fonction de génération, incluant certains des paramètres que vous avez déjà utilisés comme max_new_tokens, do_sample, top_p, top_k, temperature, num_beams, et les critères d'arrêt.

# 6 Exécution de la Génération :

Un Thread est démarré ciblant la méthode generate du modèle avec les arguments préparés dans generate_kwargs. Cette approche asynchrone permet à la fonction de fournir des messages partiels au fur et à mesure de leur génération sans attendre que le processus de génération complet soit terminé.

# 7 Fourniture de Réponses Partielles :

La fonction itère sur les nouveaux tokens du streamer. Tant que le nouveau token n'est pas le caractère spécial <, il concatène ces tokens à partial_message et fournit le message partiel. Cela permet à la fonction de fournir des mises à jour en temps réel du message en cours de génération, utile pour les applications qui veulent montrer à l'utilisateur que le bot est "en train d'écrire" ou révèle progressivement sa réponse.

# Exercise for you! / Petit exercice pour vous !

Without looking at the solution 😜, insert inside *generate_kwargs = dict( ... )*  the parameters of the model we have seen previously

---

Sans regarder la solution plus en bas 😜, mettez les paramètres que vous avez utilisés dans l'autre activité dedans *generate_kwargs = dict( ... )*

In [7]:
def predict(message, history):


    history_transformer_format = history + [[message, ""]]
    stop = StopOnTokens()

    messages = "".join(["".join(["\n<human>:"+item[0], "\n<bot>:"+item[1]])
                for item in history_transformer_format])

    model_inputs = tokenizer([messages], return_tensors="pt").to("cuda")
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        stopping_criteria=StoppingCriteriaList([stop])
        )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()



    partial_message = ""
    for new_token in streamer:
        if new_token != '<':
            partial_message += new_token
            yield partial_message



# Solution here!

I hope you were able to insert the parameters from the previous model, if not it is not a problem here you will know how to do it!

---

J'espère vous avez réussi ! et si vous avez des doutes... Voilà la solution !

In [8]:
def predict(message, history):


    history_transformer_format = history + [[message, ""]]
    stop = StopOnTokens()

    messages = "".join(["".join(["\n<human>:"+item[0], "\n<bot>:"+item[1]])
                for item in history_transformer_format])

    model_inputs = tokenizer([messages], return_tensors="pt").to("cuda")
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=80,
        do_sample=True,
        top_p=0.95,
        #top_k=1000,
        temperature=0.6,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop])
        )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    partial_message = ""
    for new_token in streamer:
        if new_token != '<':
            partial_message += new_token
            yield partial_message



# launch the chatbot!

it's time to create your chatbot. We will use the command *gr.ChatInterface* - that stands for gradio.ChatInterface that will create a simple interface for us.

We will also insert the predict function discussed above, so that when we run the app we will also run the predict function and we will begin our chat.

Note: sometimes it might display en error and this might be due to Colab, in case retry! As you see we also put *debug=True* this will help us understand most of the time the source of the error!

---

il est temps de créer votre chatbot. Nous utiliserons la commande *gr.ChatInterface* - qui représente gradio.ChatInterface et qui créera une interface simple pour nous.

Nous insérerons également la fonction predict discutée précédemment, de sorte que lorsque nous exécuterons l'application, nous exécuterons également la fonction predict et nous commencerons notre discussion.

Note : parfois, cela peut afficher une erreur et cela pourrait être dû à Colab, dans ce cas, réessayez ! Comme vous le voyez, nous mettons aussi *debug=True *cela nous aidera la plupart du temps à comprendre l'origine de l'erreur !

In [ ]:
gr.ChatInterface(predict).launch(debug=True)

# what if... I don't want other users to play with my chatbot? / Et si je voudrais décider qui peut jouer avec mon chatbot ?

Gradio allows you easily to restrict the access to the people who have the right username and password.

After the *debug=True* you can specify a parameter called *auth = ('user', 'admin')* where you can change the words "user" and "admin" to what you decide being the username and the password to enter the chatbot.

You can also add a message to the user to let them know that there is a password and username to enter to let them use the chatbot.

----

Gradio vous permet facilement de restreindre l'accès aux personnes qui ont le bon nom d'utilisateur et mot de passe.

Après le *debug=True*, vous pouvez spécifier un paramètre appelé *auth = ('user', 'admin')* où vous pouvez changer les mots "user" et "admin" par ce que vous décidez d'être le nom d'utilisateur et le mot de passe pour entrer dans le chatbot.

Vous pouvez également ajouter un message pour l'utilisateur pour les informer qu'il y a un nom d'utilisateur et un mot de passe à entrer pour leur permettre d'utiliser le chatbot.

In [ ]:
gr.ChatInterface(predict).launch(debug=True, auth = ('user','admin'), auth_message= "Enter your username and password to play with my chatbot!")